In [14]:
import pandas as pd

In [15]:
# Read sensor location file
sensor_loc = pd.read_csv("data/sensor_location.csv")

In [16]:
sensor_loc.head()

,Objectummer,Locatienaam,Lat/Long,Breedte,Effectieve breedte
0,CMSA-GAKH-01,Kalverstraat t.h.v. 1,"52.372634, 4.892071",8,"6,7"
1,CMSA-GAWW-11,Korte Niezel,"52.374616, 4.899830","3,8","3,4"
2,CMSA-GAWW-12,Oudekennissteeg,"52.373860, 4.898690",3,"2,6"
3,CMSA-GAWW-13,Stoofsteeg,"52.372439, 4.897689","2,6","2,2"
4,CMSA-GAWW-14,Oudezijds Voorburgwal t.h.v. 91,"52.373538, 4.898166",4,"3,6"


In [17]:
# Split Lat/Long into Lat and Long 
# Then store them into (new) Lat and Lon columns
sensor_loc[['Lat', 'Lon']] = sensor_loc['Lat/Long'].str.split(',', expand=True).astype(float)

In [18]:
sensor_loc.head()

,Objectummer,Locatienaam,Lat/Long,Breedte,Effectieve breedte,Lat,Lon
0,CMSA-GAKH-01,Kalverstraat t.h.v. 1,"52.372634, 4.892071",8,"6,7",52.372634,4.892071
1,CMSA-GAWW-11,Korte Niezel,"52.374616, 4.899830","3,8","3,4",52.374616,4.899830
2,CMSA-GAWW-12,Oudekennissteeg,"52.373860, 4.898690",3,"2,6",52.373860,4.898690
3,CMSA-GAWW-13,Stoofsteeg,"52.372439, 4.897689","2,6","2,2",52.372439,4.897689
4,CMSA-GAWW-14,Oudezijds Voorburgwal t.h.v. 91,"52.373538, 4.898166",4,"3,6",52.373538,4.898166


In [19]:
# Create a function to do load sensor location data 
# This function will be used in main programme 

import pandas as pd
def load_sensor_locations():
    sensor_loc = pd.read_csv("data/sensor_location.csv")
    sensor_loc[['Lat', 'Lon']] = sensor_loc['Lat/Long'].str.split(',', expand=True).astype(float)
    return sensor_loc



In [21]:
sensor_loc = load_sensor_locations()
sensor_loc.head()

,Objectummer,Locatienaam,Lat/Long,Breedte,Effectieve breedte,Lat,Lon
0,CMSA-GAKH-01,Kalverstraat t.h.v. 1,"52.372634, 4.892071",8,"6,7",52.372634,4.892071
1,CMSA-GAWW-11,Korte Niezel,"52.374616, 4.899830","3,8","3,4",52.374616,4.899830
2,CMSA-GAWW-12,Oudekennissteeg,"52.373860, 4.898690",3,"2,6",52.373860,4.898690
3,CMSA-GAWW-13,Stoofsteeg,"52.372439, 4.897689","2,6","2,2",52.372439,4.897689
4,CMSA-GAWW-14,Oudezijds Voorburgwal t.h.v. 91,"52.373538, 4.898166",4,"3,6",52.373538,4.898166


In [22]:
# Now, read sensor data
sensor_data = pd.read_csv("data/sensor_data.csv")

In [23]:
sensor_data.head()

,timestamp,CMSA-GAKH-01_0,CMSA-GAKH-01_180,CMSA-GAWW-11_120,CMSA-GAWW-11_300,CMSA-GAWW-12_115,CMSA-GAWW-12_295,CMSA-GAWW-13_120,CMSA-GAWW-13_300,CMSA-GAWW-14_40,...,GVCV-13_10,GVCV-13_190,GVCV-14_90,GVCV-14_270,hour,minute,day,month,weekday,is_weekend
0,2025-08-20 00:00:00+02:00,15,4,29,33,44,28,42,37,11,...,41,40,0,0,0,0,20,8,2,0
1,2025-08-20 00:03:00+02:00,1,3,21,29,34,39,9,14,6,...,0,0,0,0,0,3,20,8,2,0
2,2025-08-20 00:06:00+02:00,5,4,35,22,29,34,33,42,14,...,0,0,0,0,0,6,20,8,2,0
3,2025-08-20 00:09:00+02:00,4,4,40,47,42,40,19,34,15,...,0,0,0,0,0,9,20,8,2,0
4,2025-08-20 00:12:00+02:00,4,11,54,59,58,33,17,33,26,...,127,57,0,0,0,12,20,8,2,0


In [24]:
# Observation:
# Upon inspecting the 'sensor_data' and 'sensor_locations' datasets,
# the naming conventions of the sensors differ slightly.
#
# Example:
#   - In `sensor_data`:      "CMSA-GAKH-01_0", "CMSA-GAKH-01_180"
#   - In `sensor_locations`: "CMSA-GAKH-01"
#
# The suffix "_0" and "_180" in `sensor_data` indicates *bidirectional readings* 
# from the same physical sensor — one for each direction of crowd movement.

# Problem:
# This mismatch prevents direct merging or cross-referencing between the two datasets.

# Proposed Solution:
# Create a cleaned version of `sensor_data`:
#     1. Separate the base sensor name and direction.
#     2. Standardize the base name to match `sensor_locations`.
#     3. Add a new column, e.g., "direction", derived from the suffix ("_0" or "_180").
#     4. Save the result as a new file: `sensor_data_cleaned.csv`.


In [25]:
# convert column index to list
sensor_data_index_list = sensor_data.columns.tolist()
sensor_data_index_list

['timestamp',
 'CMSA-GAKH-01_0',
 'CMSA-GAKH-01_180',
 'CMSA-GAWW-11_120',
 'CMSA-GAWW-11_300',
 'CMSA-GAWW-12_115',
 'CMSA-GAWW-12_295',
 'CMSA-GAWW-13_120',
 'CMSA-GAWW-13_300',
 'CMSA-GAWW-14_40',
 'CMSA-GAWW-14_220',
 'CMSA-GAWW-15_30',
 'CMSA-GAWW-15_210',
 'CMSA-GAWW-16_30',
 'CMSA-GAWW-16_210',
 'CMSA-GAWW-17_40',
 'CMSA-GAWW-17_220',
 'CMSA-GAWW-19_115',
 'CMSA-GAWW-19_295',
 'CMSA-GAWW-20_120',
 'CMSA-GAWW-20_300',
 'CMSA-GAWW-21_120',
 'CMSA-GAWW-21_300',
 'CMSA-GAWW-23_109',
 'CMSA-GAWW-23_289',
 'GACM-04_50',
 'GACM-04_230',
 'GASA-01-A1_135',
 'GASA-01-A1_315',
 'GASA-01-A2_135',
 'GASA-01-A2_315',
 'GASA-01-B_135',
 'GASA-01-B_315',
 'GASA-01-C_135',
 'GASA-01-C_315',
 'GASA-02-01_135',
 'GASA-02-01_315',
 'GASA-02-02_135',
 'GASA-02-02_315',
 'GASA-03_105',
 'GASA-03_285',
 'GASA-04_135',
 'GASA-04_315',
 'GASA-05-O_135',
 'GASA-05-O_315',
 'GASA-05-W_135',
 'GASA-05-W_315',
 'GASA-06_95',
 'GASA-06_275',
 'GASA-06-B_95',
 'GASA-06-B_275',
 'GVCV-01_40',
 'GVCV-01_220',


In [26]:
# remove first and last 6 items because they are not sensor ids
sensor_data_index_list = sensor_data_index_list[1:-6]
sensor_data_index_list

['CMSA-GAKH-01_0',
 'CMSA-GAKH-01_180',
 'CMSA-GAWW-11_120',
 'CMSA-GAWW-11_300',
 'CMSA-GAWW-12_115',
 'CMSA-GAWW-12_295',
 'CMSA-GAWW-13_120',
 'CMSA-GAWW-13_300',
 'CMSA-GAWW-14_40',
 'CMSA-GAWW-14_220',
 'CMSA-GAWW-15_30',
 'CMSA-GAWW-15_210',
 'CMSA-GAWW-16_30',
 'CMSA-GAWW-16_210',
 'CMSA-GAWW-17_40',
 'CMSA-GAWW-17_220',
 'CMSA-GAWW-19_115',
 'CMSA-GAWW-19_295',
 'CMSA-GAWW-20_120',
 'CMSA-GAWW-20_300',
 'CMSA-GAWW-21_120',
 'CMSA-GAWW-21_300',
 'CMSA-GAWW-23_109',
 'CMSA-GAWW-23_289',
 'GACM-04_50',
 'GACM-04_230',
 'GASA-01-A1_135',
 'GASA-01-A1_315',
 'GASA-01-A2_135',
 'GASA-01-A2_315',
 'GASA-01-B_135',
 'GASA-01-B_315',
 'GASA-01-C_135',
 'GASA-01-C_315',
 'GASA-02-01_135',
 'GASA-02-01_315',
 'GASA-02-02_135',
 'GASA-02-02_315',
 'GASA-03_105',
 'GASA-03_285',
 'GASA-04_135',
 'GASA-04_315',
 'GASA-05-O_135',
 'GASA-05-O_315',
 'GASA-05-W_135',
 'GASA-05-W_315',
 'GASA-06_95',
 'GASA-06_275',
 'GASA-06-B_95',
 'GASA-06-B_275',
 'GVCV-01_40',
 'GVCV-01_220',
 'GVCV-03_42',

In [27]:
# Now time to split the strings into sensor_id and direction

sensor_info = []
for item in sensor_data_index_list:
    parts = item.split('_')
    # print(parts)
    sensor_info.append({
        'sensor_id_full':item,
        'sensor_id': parts[0],
        'sensor_direction': parts[1]
    })

sensor_info

[{'sensor_id_full': 'CMSA-GAKH-01_0',
  'sensor_id': 'CMSA-GAKH-01',
  'sensor_direction': '0'},
 {'sensor_id_full': 'CMSA-GAKH-01_180',
  'sensor_id': 'CMSA-GAKH-01',
  'sensor_direction': '180'},
 {'sensor_id_full': 'CMSA-GAWW-11_120',
  'sensor_id': 'CMSA-GAWW-11',
  'sensor_direction': '120'},
 {'sensor_id_full': 'CMSA-GAWW-11_300',
  'sensor_id': 'CMSA-GAWW-11',
  'sensor_direction': '300'},
 {'sensor_id_full': 'CMSA-GAWW-12_115',
  'sensor_id': 'CMSA-GAWW-12',
  'sensor_direction': '115'},
 {'sensor_id_full': 'CMSA-GAWW-12_295',
  'sensor_id': 'CMSA-GAWW-12',
  'sensor_direction': '295'},
 {'sensor_id_full': 'CMSA-GAWW-13_120',
  'sensor_id': 'CMSA-GAWW-13',
  'sensor_direction': '120'},
 {'sensor_id_full': 'CMSA-GAWW-13_300',
  'sensor_id': 'CMSA-GAWW-13',
  'sensor_direction': '300'},
 {'sensor_id_full': 'CMSA-GAWW-14_40',
  'sensor_id': 'CMSA-GAWW-14',
  'sensor_direction': '40'},
 {'sensor_id_full': 'CMSA-GAWW-14_220',
  'sensor_id': 'CMSA-GAWW-14',
  'sensor_direction': '220

In [28]:
sensor_info_df = pd.DataFrame(sensor_info)
sensor_info_df

,sensor_id_full,sensor_id,sensor_direction
0,CMSA-GAKH-01_0,CMSA-GAKH-01,0
1,CMSA-GAKH-01_180,CMSA-GAKH-01,180
2,CMSA-GAWW-11_120,CMSA-GAWW-11,120
3,CMSA-GAWW-11_300,CMSA-GAWW-11,300
4,CMSA-GAWW-12_115,CMSA-GAWW-12,115
...,...,...,...
69,GVCV-11_230,GVCV-11,230
70,GVCV-13_10,GVCV-13,10
71,GVCV-13_190,GVCV-13,190
72,GVCV-14_90,GVCV-14,90


In [29]:
# Merge with sensor location data
sensor_location_cleaned = pd.merge(
    sensor_info_df,
    sensor_loc,
    left_on='sensor_id',
    right_on='Objectummer',
    how='left'
)

sensor_location_cleaned

,sensor_id_full,sensor_id,sensor_direction,Objectummer,Locatienaam,Lat/Long,Breedte,Effectieve breedte,Lat,Lon
0,CMSA-GAKH-01_0,CMSA-GAKH-01,0,CMSA-GAKH-01,Kalverstraat t.h.v. 1,"52.372634, 4.892071",8,"6,7",52.372634,4.892071
1,CMSA-GAKH-01_180,CMSA-GAKH-01,180,CMSA-GAKH-01,Kalverstraat t.h.v. 1,"52.372634, 4.892071",8,"6,7",52.372634,4.892071
2,CMSA-GAWW-11_120,CMSA-GAWW-11,120,CMSA-GAWW-11,Korte Niezel,"52.374616, 4.899830","3,8","3,4",52.374616,4.899830
3,CMSA-GAWW-11_300,CMSA-GAWW-11,300,CMSA-GAWW-11,Korte Niezel,"52.374616, 4.899830","3,8","3,4",52.374616,4.899830
4,CMSA-GAWW-12_115,CMSA-GAWW-12,115,CMSA-GAWW-12,Oudekennissteeg,"52.373860, 4.898690",3,"2,6",52.373860,4.898690
...,...,...,...,...,...,...,...,...,...,...
69,GVCV-11_230,GVCV-11,230,GVCV-11,NDSMkade,"52.401060, 4.891368","3,5","3,1",52.401060,4.891368
70,GVCV-13_10,GVCV-13,10,GVCV-13,Buiksloterweg,"52.382352, 4.903016","3,5","3,1",52.382352,4.903016
71,GVCV-13_190,GVCV-13,190,GVCV-13,Buiksloterweg,"52.382352, 4.903016","3,5","3,1",52.382352,4.903016
72,GVCV-14_90,GVCV-14,90,GVCV-14,Buiksloterweg,"52.382169, 4.903385","3,5","3,1",52.382169,4.903385


In [30]:
# Save as CSV
sensor_location_cleaned.to_csv("data/sensor_location_cleaned.csv", index=False)

In [32]:
# Update load_sensor_locations() to read from cleaned dataset instead

def load_sensor_locations():
    sensor_loc = pd.read_csv("data/sensor_location_cleaned.csv")
    # Split Lat/Long into floats
    sensor_loc[['Lat', 'Lon']] = sensor_loc['Lat/Long'].str.split(',', expand=True).astype(float)
    return sensor_loc
sensor_loc = load_sensor_locations()
sensor_loc.head()


,sensor_id_full,sensor_id,sensor_direction,Objectummer,Locatienaam,Lat/Long,Breedte,Effectieve breedte,Lat,Lon
0,CMSA-GAKH-01_0,CMSA-GAKH-01,0,CMSA-GAKH-01,Kalverstraat t.h.v. 1,"52.372634, 4.892071",8,"6,7",52.372634,4.892071
1,CMSA-GAKH-01_180,CMSA-GAKH-01,180,CMSA-GAKH-01,Kalverstraat t.h.v. 1,"52.372634, 4.892071",8,"6,7",52.372634,4.892071
2,CMSA-GAWW-11_120,CMSA-GAWW-11,120,CMSA-GAWW-11,Korte Niezel,"52.374616, 4.899830","3,8","3,4",52.374616,4.899830
3,CMSA-GAWW-11_300,CMSA-GAWW-11,300,CMSA-GAWW-11,Korte Niezel,"52.374616, 4.899830","3,8","3,4",52.374616,4.899830
4,CMSA-GAWW-12_115,CMSA-GAWW-12,115,CMSA-GAWW-12,Oudekennissteeg,"52.373860, 4.898690",3,"2,6",52.373860,4.898690


In [33]:
# Get crowd count 

for i,row in sensor_loc.iterrows():
    sensor_id = row['sensor_id_full']
    crowd_count = sensor_data[sensor_id][0]
    print(crowd_count)

15
4
29
33
44
28
42
37
11
3
21
26
73
41
19
28
27
31
27
50
33
45
12
13
20
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
14
2
0
0
0
55
59
41
40
0
0
